<a href="https://colab.research.google.com/github/joseluis-martin/Project-Dendjet/blob/main/Maderas_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importaciones

Este bloque inicial carga todas las "herramientas" o librerías que el programa necesita para funcionar.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models # Importamos 'models' para las CNNs
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import time
import os

print("PyTorch Version:", torch.__version__)

* `torch, torch.nn, torch.optim`: Forman el núcleo de PyTorch. torch se encarga de los tensores (la estructura de datos fundamental), torch.nn (Neural Networks) proporciona las capas de la red neuronal (como la capa de clasificación), y torch.optim contiene los optimizadores (como AdamW) que ajustan el modelo.

* `torchvision`: Es la librería de PyTorch para tareas de visión por computador. datasets nos da acceso a ImageFolder, transforms nos permite procesar las imágenes, y models contiene arquitecturas clásicas.

* `DataLoader`: Una utilidad clave de PyTorch que carga los datos en lotes (batches), los mezcla aleatoriamente (shuffle) y puede usar múltiples núcleos de CPU (num_workers) para cargar datos eficientemente.

* `numpy`: Una librería para computación numérica. La usamos para convertir los tensores de PyTorch a un formato que scikit-learn pueda entender.

* `matplotlib.pyplot` y `seaborn`: Son librerías de visualización. matplotlib es la base para crear gráficos, y seaborn nos permite crear gráficos estadísticos más atractivos, como el mapa de calor de la matriz de confusión.

* `sklearn.metrics`: Parte de Scikit-learn, nos proporciona herramientas ya hechas para evaluar el rendimiento del modelo, como classification_report (precisión, recall, f1-score), confusion_matrix, y accuracy_score.

* `time` y `os`: Utilidades estándar de Python. time nos permite medir cuánto tarda cada época de entrenamiento, y os nos ayuda a interactuar con el sistema operativo (por ejemplo, para crear carpetas).

## 2. Configuración y Parámetros ⚙️

Esta sección centraliza todas las variables y parámetros que podrías querer ajustar. Tenerlos aquí al principio facilita la experimentación.

In [ ]:
# ¡¡IMPORTANTE!! Ajusta estas rutas a nuestro sistema
TRAIN_DIR = '/content/drive/MyDrive/Proyecto_Maderas_Sarcofagos/dataset/train'
VAL_DIR = '/content/drive/MyDrive/Proyecto_Maderas_Sarcofagos/dataset/val'
TEST_DIR = '/content/drive/MyDrive/Proyecto_Maderas_Sarcofagos/dataset/test'

# Parámetros del entrenamiento
NUM_EPOCHS = 15
BATCH_SIZE = 32
LEARNING_RATE = 1e-4

# Configuración del dispositivo (usar GPU si está disponible)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")


* `TRAIN_DIR y VAL_DIR`: Son cadenas de texto que contienen la ruta a tus carpetas de entrenamiento y validación.

* `NUM_EPOCHS`: El número de veces que el modelo verá el conjunto de datos de entrenamiento completo.

* `BATCH_SIZE`: El número de imágenes que el modelo procesa a la vez antes de actualizar sus pesos. Un batch size más grande puede acelerar el entrenamiento pero consume más memoria de la GPU.

* `LEARNING_RATE`: La "tasa de aprendizaje". Controla el tamaño de los pasos que da el optimizador para corregir los errores. Es uno de los hiperparámetros más importantes.

* `DEVICE`: Este código es "agnóstico al dispositivo". Comprueba si tienes una GPU con CUDA disponible (torch.cuda.is_available()) y la selecciona. Si no, usará la CPU. Entrenar en GPU es miles de veces más rápido.

## 3. Preparación de Datos 💾

Este bloque se encarga de definir cómo se deben cargar y transformar las imágenes para que sean aptas para el modelo.

In [ ]:
# Transformaciones de datos
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Crear datasets
try:
    train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transforms)
    test_dataset = datasets.ImageFolder(TEST_DIR, transform=val_transforms)
except FileNotFoundError:
    print("Error: Las carpetas del dataset no se encontraron. Por favor, actualiza las rutas.")
    # Crea datos dummy para que el script no falle en un entorno de prueba
    for dir_path in [f"{TRAIN_DIR}/dummy_class", f"{VAL_DIR}/dummy_class", f"{TEST_DIR}/dummy_class"]:
        if not os.path.exists(dir_path): os.makedirs(dir_path)
    from PIL import Image
    dummy_img = Image.new('RGB', (100, 100), color='blue')
    dummy_img.save(f"{TRAIN_DIR}/dummy_class/dummy.png")
    dummy_img.save(f"{VAL_DIR}/dummy_class/dummy.png")
    dummy_img.save(f"{TEST_DIR}/dummy_class/dummy.png")
    train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
    val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transforms)
    test_dataset = datasets.ImageFolder(TEST_DIR, transform=val_transforms)

# Crear DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Obtener nombres y número de clases
CLASS_NAMES = train_dataset.classes
NUM_CLASSES = len(CLASS_NAMES)
print(f"Dataset encontrado. Clases: {CLASS_NAMES}")


* `transforms.Compose([...])`: Crea una secuencia de transformaciones que se aplicarán a cada imagen.

* `train_transforms`: Para los datos de entrenamiento, aplicamos aumento de datos (RandomHorizontalFlip, RandomRotation, ColorJitter) para crear variaciones artificiales y forzar al modelo a aprender las características esenciales de la madera, no la orientación o iluminación específica de una foto.

* `val_transforms`: Para la validación, no aplicamos aumento de datos, solo los cambios necesarios (Resize, ToTensor, Normalize) para que las imágenes tengan el formato correcto. Queremos evaluar el modelo en las imágenes originales, sin alterar.

* `datasets.ImageFolder(...)`: Esta es la función mágica que crea el dataset. Recorre las carpetas TRAIN_DIR y VAL_DIR, asume que cada subcarpeta es una clase, y asigna etiquetas numéricas automáticamente.

* `DataLoader(...)`: Envuelve el dataset y nos permite iterar sobre él en lotes (batch_size) de manera eficiente.

* `CLASS_NAMES y NUM_CLASSES`: Extraemos los nombres de las clases (los nombres de las carpetas) y el número total de clases directamente del dataset. Esto hace que el código sea adaptable a cualquier número de maderas que quieras clasificar.

## 4. Definición del del Modelo CNN (ResNet50) 🧠

Aquí es donde se define y adapta la arquitectura de la red neuronal.



In [ ]:
# Cargar una arquitectura EfficientNet-B1 con pesos pre-entrenados
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)

# En EfficientNet, la capa de clasificación es parte de una secuencia llamada 'classifier'.
# La capa lineal que queremos reemplazar es la última de esta secuencia.
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, NUM_CLASSES)

# Mover el modelo a la GPU
model.to(DEVICE)

# Definir función de pérdida y optimizador (sin cambios)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

* `models.`: Esta línea carga la arquitectura red correspondiente e inicializa sus pesos con los que aprendió en el gigantesco dataset ImageNet. Esto significa que el modelo ya es un experto en reconocer patrones, texturas y formas visuales.

* `num_ftrs = model.fc.in_features`: El código inspecciona la última capa del modelo pre-entrenado (llamada fc) para ver cuántas neuronas de entrada tiene.

* `model.fc = nn.Linear(num_ftrs, NUM_CLASSES)`: Esta es la adaptación clave. Se reemplaza la última capa original (que estaba diseñada para clasificar 1000 objetos de ImageNet) por una nueva capa lineal. Esta nueva capa tiene el número correcto de salidas para corresponder a nuestras clases de madera.

* `criterion y optimizer`: Se definen la función de pérdida (CrossEntropyLoss), que mide el error del modelo, y el optimizador (AdamW), que se encarga de minimizar ese error ajustando los pesos del modelo.

## 5. Función de Evaluación

Esta es una función auxiliar que hemos creado para mantener el código limpio. Su única responsabilidad es medir el rendimiento del modelo en un conjunto de datos, sin entrenarlo.

In [ ]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = accuracy_score(all_labels, all_preds)
    return epoch_loss, epoch_acc, all_preds, all_labels

* `model.eval()`: Pone el modelo en modo evaluación (desactiva Dropout, etc.).

* `with torch.no_grad()`: Desactiva el cálculo de gradientes para acelerar el proceso y ahorrar memoria, ya que aquí no vamos a entrenar.

* El resto del código itera sobre los datos, calcula la pérdida y la precisión, y devuelve estos valores.

## 6. Bucle Principal de Entrenamiento y Validación 🏋️‍♂️

Este es el motor del script, donde el aprendizaje realmente ocurre. Itera varias veces (épocas) sobre los datos.

In [ ]:
history = {'train_loss': [], 'val_loss': [], 'val_accuracy': []}

print("\n--- Iniciando Entrenamiento ---")
for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    # Fase de Entrenamiento
    model.train()
    running_train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item() * inputs.size(0)
    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    history['train_loss'].append(epoch_train_loss)

    # Fase de Validación
    val_loss, val_acc, _, _ = evaluate_model(model, val_loader, criterion, DEVICE)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)

    epoch_duration = time.time() - start_time
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Duración: {epoch_duration:.2f}s | Train Loss: {epoch_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

print("\n--- Entrenamiento Completado ---")

* Fase de Entrenamiento (model.train()):

    * `optimizer.zero_grad()`: Borra los gradientes del paso anterior.

    * `outputs = model(inputs)`: Pasa un lote de imágenes por la red para obtener las predicciones.

    * `loss.backward()`: Calcula cómo contribuyó cada peso del modelo al error (backpropagation).

    * `optimizer.step()`: Actualiza los pesos del modelo para reducir el error.

* Fase de Validación: Después de cada época de entrenamiento, se llama a la función evaluate_model sobre los datos de validación. Esto nos permite monitorizar si el modelo está aprendiendo correctamente y no se está sobreajustando.

## 7. Evaluación Final y Visualización 📊

Una vez que el entrenamiento ha terminado, esta sección final proporciona un análisis detallado y visual del rendimiento del mejor modelo.

In [ ]:
print("\n--- Realizando Evaluación Final sobre el Conjunto de Prueba ---")
test_loss, test_acc, test_preds, test_labels = evaluate_model(model, test_loader, criterion, DEVICE)
print(f"Resultado Final: Pérdida de Prueba: {test_loss:.4f} | Precisión de Prueba: {test_acc:.4f}\n")

# Reporte de Clasificación
print("--- Reporte de Clasificación (Test Set) ---")
print(classification_report(test_labels, test_preds, target_names=CLASS_NAMES, zero_division=0))

# Matriz de Confusión
print("\n--- Matriz de Confusión (Test Set) ---")
conf_matrix = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES)
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Real')
plt.title('Matriz de Confusión sobre el Conjunto de Prueba')
plt.show()

# Gráficas del historial de entrenamiento
print("\n--- Visualización del Historial de Entrenamiento ---")
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Pérdida de Entrenamiento')
plt.plot(history['val_loss'], label='Pérdida de Validación')
plt.xlabel('Épocas'); plt.ylabel('Pérdida'); plt.title('Evolución de la Pérdida'); plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history['val_accuracy'], label='Precisión de Validación')
plt.xlabel('Épocas'); plt.ylabel('Precisión'); plt.title('Evolución de la Precisión'); plt.legend()
plt.tight_layout()
plt.show()

* `evaluate_model(model, test_loader, ...)`: Se llama a la función de evaluación una última vez, pero con los datos de prueba.

* `classification_report`: Muestra métricas detalladas como la precisión y el recall para cada clase de madera, permitiéndonos ver si el modelo es mejor reconociendo una especie que otra.

* `confusion_matrix`: Se visualiza como un mapa de calor. La diagonal principal muestra los aciertos. Los números fuera de la diagonal muestran los errores de clasificación (por ejemplo, cuántas veces confundió "cedro" con "acacia").

* Gráficas de Historial: Se dibujan las curvas de pérdida y precisión a lo largo de las épocas. Estos gráficos son vitales para diagnosticar cómo fue el entrenamiento y para detectar problemas como el sobreajuste.